In [1]:
import math
from tensorboardX import SummaryWriter
import roboschool
import quadruppedEnv
import env_configurations
sess = None

robot lib not fouund

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)

config=tf.ConfigProto(gpu_options=gpu_options)
sess = tf.InteractiveSession(config=config)

In [3]:
import gym
import wrappers

env_name = 'RoboschoolHumanoid-v1'#'QuadruppedWalk-v1' #'QuadruppedWalk-v1' #$'BipedalWalker-v2'#'RoboschoolAnt-v1'
#'BipedalWalker-v2'#'RoboschoolAnt-v1' #'LunarLanderContinuous-v2'#'BipedalWalker-v2'#'LunarLander-v2' 
#'MountainCarContinuous-v0'#'CarRacing-v0'#'CartPole-v1' #'RoboschoolAnt-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
env = env_configurations.a2c_configurations[env_name]['ENV_CREATOR']()
print(env.action_space.low)
print(env.action_space.high)

[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
import ray
ray.init(redis_max_memory=1024*1024*100, object_store_memory=1024*1024*100)

2019-05-27 15:38:43,180	INFO node.py:469 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-27_15-38-43_19137/logs.
2019-05-27 15:38:43,292	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:43905 to respond...
2019-05-27 15:38:43,440	INFO services.py:407 -- Waiting for redis server at 127.0.0.1:45193 to respond...
2019-05-27 15:38:43,448	INFO services.py:804 -- Starting Redis shard with 0.1 GB max memory.
2019-05-27 15:38:43,515	INFO node.py:483 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-27_15-38-43_19137/logs.
2019-05-27 15:38:43,517	INFO services.py:1427 -- Starting the Plasma object store with 0.1 GB memory using /dev/shm.


{'node_ip_address': '192.168.3.104',
 'redis_address': '192.168.3.104:43905',
 'object_store_address': '/tmp/ray/session_2019-05-27_15-38-43_19137/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-05-27_15-38-43_19137/sockets/raylet',
 'webui_url': None}

In [5]:
from a2c_v2 import A2CAgent
import tr_helpers
import networks
import a2c_games_configurations

a2c_config = a2c_games_configurations.roboschoolhumanoid_config
#a2c_config = a2c_games_configurations.roboschoolant_config
#self, sess, env_name, observation_shape, actions_num, config = default_config
observation_shape = env.observation_space.shape
action_space = env.action_space
print(observation_shape)
agent = A2CAgent(sess,'rob_20', env.observation_space, False, action_space, a2c_config)
agent.restore('nn/latest')
#agent.train()


(44,)
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


2019-05-27 15:38:44,816	ERROR worker.py:1672 -- WARNING: 24 workers have been started. This could be a result of using a large number of actors, or it could be a consequence of using nested tasks (see https://github.com/ray-project/ray/issues/3644) for some a discussion of workarounds.


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from nn/latest


In [6]:
ray.shutdown()

In [14]:
import numpy as np
#agent.restore('nn/latest')
#agent.restore('nn/rob_20QuadruppedWalk-v1')
#agent.restore('nn/ppo_continuousQuadruppedWalk-v1')

def evaluate(env,t_max=5000):
    rewards = []
    env._max_episode_steps = 5000
    env.batch_mode = False
    print('reset')
    #env = env.old_env
    s = env.reset()
    reward = 0
    for it in range(5000):
        action = agent.get_action([s], False)
        
        #ad = agent.get_action_distribution([s])
        #print(ad)
        s, r, done, _ = env.step(np.squeeze(action, axis = 0))
        print(action)
        env.render()
        reward += r
        
            
        if done:
            break       
        
    return reward

import gym.wrappers

#env_monitor = wrappers.make_atari_deepmind(env_name, noop_max=30, skip=4)
#env_monitor = wrappers.ReallyDoneWrapper(env_monitor)
#env_monitor = gym.wrappers.Monitor(env,directory='video_ppo',force=True)

sessions = [print('reward:', evaluate(env)) for _ in range(1)]
#env_monitor.close()




reset
[[-0.8255239  -0.17956099  0.7214522   0.9877366  -0.27738094  0.71427625
  -0.9056543   0.7479178   0.3310758  -0.21746825 -0.212265   -1.
  -0.06286152 -0.4769557   1.          0.6674597  -0.11961802]]
[[-0.6973046   0.25006807  0.43778723  0.98883283 -0.01387381  0.47475237
  -0.4211074   0.7611384   0.01747885 -0.5260117  -0.3233176  -1.
   0.05424889 -0.0233732   0.80706275  0.48344895  0.10759865]]
[[-0.52106714  0.47688463 -0.17570275  1.          0.10786754  0.5276215
  -0.00415955  0.66803455 -0.05524714 -0.48400697 -0.29142603  1.
   0.34300026 -0.1423426  -1.          0.5167107  -0.03786893]]
[[-0.10896721  0.4922015  -0.09316108  1.          0.33249754  0.2588851
  -0.2004694   0.48539406 -0.17741671 -0.18996575 -0.03717345  1.
   0.2109038   0.04350597 -1.          0.49636424  0.03526854]]
[[-0.59087175 -0.22004469  0.00827653 -1.          0.06339778  0.00926635
   0.29296023  0.661785   -0.20440502 -0.09152324 -0.1251802   1.
   0.43136865 -0.00369021 -1.         -0

[[ 0.11947674 -0.18997462 -0.1446034  -1.          0.21419163 -0.25938383
   0.763733    0.5777282   0.5628922   0.02404837 -0.30545193 -1.
   0.14638534  0.22088891  1.          0.2779945   0.09183606]]
[[-0.1400607   0.20640868 -0.26702827  1.          0.09251736 -0.16930175
   0.77804595  0.30828023  0.2595357  -0.00567822 -0.03188962 -1.
  -0.46635574 -0.16971347  1.          0.05383344  0.24035823]]
[[-0.29532254 -0.3635546   0.02976082 -1.          0.281987    0.07639769
   0.7482035   0.20207478 -0.49328727  0.11141246 -0.6605197   1.
   0.07766837 -0.02581418 -1.         -0.17146233 -0.11673269]]
[[ 0.28999692 -0.2974682   0.276926   -1.         -0.09878048 -0.00544079
   0.7565591   0.10209516  0.10363635 -0.36968854  0.0153369   1.
   0.2896037  -0.01782372  1.          0.03822578  0.05284937]]
[[ 0.23350783  0.2657968   0.33829993  1.          0.26057357 -0.04267906
   0.59058386  0.54565406  0.33221453 -0.10129448 -0.20491345  1.
   0.45988494  0.0768621   1.          0.364

[[-0.00790393 -0.35278845  0.37926698  1.          0.20489873  0.20012622
  -0.07064977 -0.34607142 -0.5894332  -0.4099705  -0.524836   -1.
   0.18471998 -0.12152924 -1.         -0.07545954  0.20026186]]
[[ 0.13794848  0.24163124  0.78600603 -1.          0.06140474  0.08350946
   0.02310091 -0.5883416   0.06174633 -0.22274563  0.1255685   1.
  -0.9171345  -0.18683784 -1.         -0.50477225 -0.0084768 ]]
[[-0.0068883  -0.00814193  0.332146    0.28639698  0.46811503  0.31694394
  -0.15841445 -0.4488874  -0.12291596 -0.42173794  0.01661137  1.
  -0.08321685 -0.08535551 -1.         -0.35888445  0.05880727]]
[[ 0.10127342 -0.1410108   0.5598403   0.7584455   0.5307786   0.10022513
   0.7039145  -0.742495   -0.06965344 -0.5373736   0.03675456  1.
  -0.57410306  0.30903083  1.         -0.14231822 -0.38020688]]
[[ 0.25951326 -0.39736128  0.15174961  1.         -0.01039791 -0.11875933
   0.29949117 -0.66057324  0.2571019  -0.37359527  0.4778005  -1.
  -0.38730848  0.03436209 -1.         -0.232

[[-0.25498536 -0.00130557  0.08435188 -1.          0.20016372 -0.20887108
   0.85804904 -0.5139483  -0.10159004  0.02614973 -0.00762476  1.
  -0.3917269   0.09043283 -1.          0.0328421   0.11486781]]
[[ 0.4551498   0.34633946  0.1917235   1.         -0.11537616 -0.34597018
   0.51483506 -0.20750979  0.24307905 -0.18388578 -0.1519631   1.
   0.23506916 -0.13920364  1.         -0.41983637  0.01222327]]
[[ 0.04874949 -0.23439442 -0.30467457  1.          0.08050147 -0.33539337
   0.7096243   0.62048286 -0.40384263 -0.0554511  -0.04877242  1.
  -0.5833351   0.06913589  1.         -0.3512214   0.07491373]]
[[-0.20302373 -0.6518964   0.2306469   1.         -0.01142693  0.15985212
   0.33226967  0.09692397  0.327337    0.0537094  -0.33545655  1.
   0.27855408 -0.10372604  1.          0.28362578  0.04870603]]
[[ 0.5792975   0.05936738  0.44173658 -1.          0.07704696 -0.02814367
   0.5076489   0.6125484   0.17376333  0.06433668 -0.03426271  1.
  -0.33361968 -0.00295314 -1.          0.089

[[ 0.03386126  0.39435178 -0.43051827 -1.         -0.01397967  0.20633423
  -0.14948359 -0.2367302  -0.5210676  -0.01808409 -0.11241093 -1.
  -0.27366388 -0.19298044  1.         -0.2139506   0.27379435]]
[[-0.5251651  -0.2171183   0.30031097  1.         -0.00948297  0.23003465
   0.3058536  -0.30736354 -0.1780076  -0.60852677 -0.56881976  1.
  -0.7319613  -0.05229972  1.         -0.13286722  0.06709638]]
[[-0.18104318 -0.61724395  0.88180345 -1.          0.13361317  0.16703828
  -0.02371045 -0.18465936 -0.06144069 -0.5506608   0.0215315   1.
   0.3414414  -0.02475314 -1.         -0.44088614  0.10303774]]
[[ 0.37229577 -0.5135722   0.8711904   0.11132753  0.04973155  0.11545937
   0.21502508 -0.2009131  -0.14979023 -0.73976344  0.02263371 -1.
   0.26985466  0.01705564  1.         -0.05882806  0.06748244]]
[[ 0.07304008 -0.14956406  0.99727815 -1.          0.06118887  0.4752859
  -0.02873122 -0.36010477 -0.01400094 -0.36907387  0.1512864  -1.
  -0.10616128 -0.14267686  1.         -0.3931

...

[[-0.12778288  0.3591573  -0.5712823   1.          0.19186835 -0.5423185
   0.37486383  0.22462827  0.3614231   0.9555544   0.3638522   1.
   0.7632822  -0.03425421  1.          0.19294235  0.25569952]]
[[-0.6072731  -0.5817594  -0.41807336 -1.         -0.29058567 -0.16227399
  -0.04622175 -0.375981    0.39035657  0.974446    0.26239973 -1.
   0.61869395  0.20681885 -1.          0.19883804 -0.06431395]]
[[-0.6810075   0.98330873 -1.         -1.         -0.333321   -0.529998
   0.21374592 -0.17959222  0.5500341   0.8597315   0.5327537  -1.
  -0.48803455 -0.02027997 -1.          0.47603917  0.0350934 ]]
[[ 0.32008094  0.06628874 -0.28385404 -1.         -0.24082413  0.02673395
   0.20055136  0.1152286   0.54706514  1.          0.66186756 -1.
   1.          0.00768481  1.         -0.3345373   0.04377531]]
[[-0.61036766  0.72764593 -0.37941197 -1.         -0.30532902 -0.62278974
   0.2332665  -0.5825738   0.4189584   0.9932736   0.38578796 -1.
   0.23635693 -0.23930633  1.          0.303099

[[ 0.2741128  -0.01161087  0.7394012  -1.         -0.14069524  0.07476924
   0.05199628  0.03929867  0.21427032 -0.1564423  -0.04137588 -1.
  -0.11875814 -0.1254398   1.          0.4783572   0.1258364 ]]
[[-0.36988974 -0.08619584  0.6639191   1.         -0.0381808   0.06827171
   0.28775758  0.10797679  0.10104574  0.00171325 -0.03775761 -1.
   0.29867825  0.02085312 -1.          0.16841467  0.08271255]]
[[-0.5312104   0.30530918  0.69886464  1.         -0.58211    -0.00557088
   0.64153516  0.5194184   0.1308674  -0.18954483  0.03454738 -1.
   0.24152586 -0.11091641 -1.         -0.5511794   0.06321694]]
[[-0.48709732  0.63430536  0.7122885   1.          0.02535477  0.13604632
   0.5782196   0.8938892   0.39558917  0.0445034  -0.08911766  1.
   0.33017725 -0.06109766 -1.          0.11699742  0.10110856]]
[[-0.23134735  0.08294629  0.71111476  1.          0.08775043 -0.07951565
   0.13707738  0.7930741   0.06182215  0.22920454 -0.10180733  1.
   0.24246171 -0.00743289 -1.          0.044

[[ 0.29302502  1.          0.0825595  -1.         -0.0584034  -0.23352146
   0.02970585 -1.         -0.404749    0.97354084  0.5936868  -1.
   0.41389963 -0.12860121 -1.          0.26978528 -0.20720264]]
[[-0.20264262  0.81516874  0.17364573 -1.          0.27510816 -0.2567444
   0.22563264 -0.3697765  -0.3480992   0.9620489   0.3672734   1.
   0.1266712  -0.06991304  1.          1.         -0.00445392]]
[[ 0.35675398  0.10087883  0.19211835 -1.         -0.04327351  0.10469
   0.0451544  -0.3010979  -0.65285426  0.993418    0.8006925  -1.
  -1.         -0.05418725  1.         -0.48680252  0.02768191]]
[[-0.43781686  0.19410568 -0.8389541  -1.         -0.23923153 -0.23756817
   0.18919638 -0.64947665 -0.3598938   1.          0.7197589   1.
  -0.36212882 -0.06988913  1.         -0.20599039  0.18381493]]
[[-0.25373003  0.8099814  -1.         -1.         -0.11563921 -0.2880773
   0.05522571 -0.59005237 -0.8359247   0.8975054   0.77749866 -1.
   0.0552614  -0.03693129  1.          0.5037168 

[[ 0.4541606   0.7376074  -0.64356697  0.666754    0.31625643 -0.68188715
   1.          0.3035975   0.17056556 -0.045691    0.11095917 -1.
  -0.7977781  -0.30990696  1.         -0.4320277  -0.1187098 ]]
[[ 0.52066255  0.24466257 -0.8104722  -1.         -0.12964505 -0.13441299
   0.8320192   0.46795267  0.48632818 -0.168417    0.07757646 -1.
  -0.8620405  -0.08783937  1.         -0.5731021  -0.03524164]]
[[ 0.18285361  0.6125576  -0.7952539   1.          0.13732834 -0.34995365
   0.810212    0.53370845  0.30686226 -0.09599505  0.40078488  1.
   0.01728454 -0.14183801  1.         -0.2979164  -0.19896452]]
[[ 0.8105373   0.26377335 -0.7786782   1.          0.09979937  0.16625223
   0.8225834   0.5448483   0.30984718  0.4800369  -0.09319317  1.
  -1.         -0.21252492 -1.         -0.6912092  -0.2570432 ]]
[[ 0.18800251  0.5732332  -0.6756102  -1.          0.07420465 -0.46347484
   1.          0.12663245  0.08199453  0.30834588  0.8132069  -1.
  -0.38484615 -0.1606554   1.          0.027

[[ 1.          0.5385742  -0.9721456   1.         -0.19201705  0.03851053
  -0.39424777  0.59141576 -0.69999325  0.03675085  0.3848804  -1.
   1.          0.19145235 -1.          0.16308844 -0.09999432]]
[[ 0.6569995   0.0642405  -0.62224114  1.          0.41978988  0.07994696
  -0.5611712  -0.47278467  0.59794015  0.72787553  0.06853321  1.
   0.67673355 -0.24161792 -1.          0.78094864  0.02832513]]
[[-1.          0.2674547  -1.          1.          0.65253586 -0.20149457
   0.39734182 -1.          1.          0.73072857  0.11064316  1.
  -0.63758826  0.3156318  -1.          0.13532695  0.01603106]]
[[-0.17799827 -0.06756544 -0.54410416  1.          0.13190553 -0.40820196
   0.25966322  1.         -0.38425708  1.          0.56357026  1.
  -0.00847711 -0.7558283  -1.          0.2279741   0.43523493]]
[[-0.35357642  0.10670248 -0.23108202  1.          0.6373108   0.2987646
  -0.2827506   0.55623364  0.20710242  0.7405831   0.30246347 -1.
  -0.77410525 -0.18337673  1.         -0.0978

In [8]:
obs = env.reset()
obs, r, done, _ = env.step(2)

done = False
for _ in range(130):
    obs, _, done, _ = env.step(1)
    #env.render()
    obs = np.array(obs)
    if done:
        print('done')
        break
plt.title("what your network gonna see")
print (obs.shape)
plt.imshow(obs[:,:,0],interpolation='none',cmap='gray');

TypeError: 'int' object is not subscriptable

In [ ]:
import gym
gym.envs.registry.all()